In [ ]:
pip install openai

In [ ]:
pip install dotenv

In [24]:
import os
from openai import OpenAI

In [25]:
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
token = os.getenv('GITHUB_TOKEN')
endpoint = "https://models.inference.ai.azure.com"
model_name = "text-embedding-3-small"

In [ ]:
print(token)

In [27]:
client = OpenAI(
    base_url=endpoint,
    api_key=token,
)


In [28]:
#Input a fixed list of phrases to get their embeddings

response = client.embeddings.create(
    input=["first phrase", "second phrase", "third phrase"],
    model=model_name,
)

for item in response.data:
    length = len(item.embedding)
    print(
        f"data[{item.index}]: length={length}, "
        f"[{item.embedding[0]}, {item.embedding[1]}, "
        f"..., {item.embedding[length-2]}, {item.embedding[length-1]}]"
    )
print(response.usage)

data[0]: length=1536, [-0.00721184303984046, 0.007491494063287973, ..., 0.01611734740436077, -0.004887983202934265]
data[1]: length=1536, [-0.003025691257789731, 0.009231699630618095, ..., 0.029947662726044655, 0.020937401801347733]
data[2]: length=1536, [-0.013795719482004642, 0.031857650727033615, ..., 0.017506178468465805, 0.0226223636418581]
Usage(prompt_tokens=6, total_tokens=6)


To extract data from PDF

In [ ]:
pip install chromadb

In [ ]:
pip install pymupdf

In [ ]:
pip install langchain-openai

In [ ]:
pip install langchain-community

In [30]:
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

In [31]:
dir="docs/"

In [33]:
#Loading the documents from the directory

def load_docs(dir):
    loader=DirectoryLoader(dir,loader_cls=PyMuPDFLoader,use_multithreading=True,max_concurrency=128,show_progress=True,silent_errors=True)
    documents=loader.load()
    return documents

In [32]:
#Splitting the documents into chunks

def split_docs(documents,chunk_size=1000,chunk_overlap=100):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(documents)
    return docs

In [34]:
documents=load_docs(dir)
len(documents)

100%|██████████| 1/1 [00:00<00:00, 30.30it/s]


6

In [35]:
doc=split_docs(documents)
print(len(doc))

12


In [36]:
#langchain client OpenAIEmbeddings

clientopen = OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url=endpoint,
    api_key=token,
)

In [37]:
save_to=Chroma.from_documents(documents=doc,embedding=clientopen,persist_directory='./ai-toolkit')

In [38]:
query="What is AI Toolkit?"

In [39]:
db1=Chroma(persist_directory='./ai-toolkit',embedding_function=clientopen)
results=db1.similarity_search(query)
print(results)
print(results[0].page_content)

[Document(metadata={'author': 'Shreyan Fernandes', 'creationDate': "D:20240708201313+05'30'", 'creationdate': '2024-07-08T20:13:13+05:30', 'creator': 'Microsoft® Word 2019', 'file_path': 'docs\\AIToolkit.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20240708201313+05'30'", 'moddate': '2024-07-08T20:13:13+05:30', 'page': 0, 'producer': 'Microsoft® Word 2019', 'source': 'docs\\AIToolkit.pdf', 'subject': '', 'title': '', 'total_pages': 6, 'trapped': ''}, page_content='The AI Toolkit for Visual Studio Code (VS Code) was previously known as Windows AI Studio. The \nextension has been renamed to reflect the focus on enabling AI development in VS Code across \nplatforms. \n \nThe AI Toolkit for VS Code (AI Toolkit) is a VS Code extension that enables you to: \n \nDownload and run AI models locally. The AI Toolkit provides out-of-the-box access to highly \noptimized models for the following platforms and hardware: \n• \nWindows 11 running with DirectML acceleration \n• \nWindows 11 